In [ ]:
using Colors
using SparseArrays
using LinearAlgebra
using CSV
using Plots


include("src/utils/helper.jl")


In [ ]:
results_0 = csv_to_results("/home/dro/workspace-eric/Efficient-GLP/run_results/a9a-clvr_lazy_restart_x_y-blocksize=1-R=1-2021-10-21_22-02-26-741.csv")
results_1 = csv_to_results("/home/dro/workspace-eric/Efficient-GLP/run_results/a9a-iclr_lazy_restart_x_y-2021-10-17_06-00-22-316.csv")
results_2 = csv_to_results("/home/dro/workspace-eric/Efficient-GLP/run_results/a9a-pdhg_restart_x_y-2021-10-17_21-29-41-438.csv")
results_3 = csv_to_results("/home/dro/workspace-eric/Efficient-GLP/run_results/a9a-spdhg_restart_x_y-2021-10-17_06-00-28-860.csv")
results_4 = csv_to_results("/home/dro/workspace-eric/Efficient-GLP/run_results/a9a-purecd_restart_x_y-2021-10-21_17-52-17-998.csv")


-----

In [ ]:
DATASET_INFO = Dict([
    ("a1a", (123, 1605)),
    ("a9a", (123, 32561)),
    ("gisette", (5000, 6000)),
    ("news20", (1355191, 19996)),
    ("rcv1", (47236, 20242)),
])


dataset = "a9a"
d, n = DATASET_INFO[dataset]



figuretitle = "$(dataset) (n = $(n), d = $(d))"


savedir = "./final_plots"

markerstep = 150

xtickfontsize = 10
ytickfontsize = 10
xguidefontsize = 13
yguidefontsize = 13
legendfontsize = 12


plotconfig = Dict(
    :xtickfontsize => xtickfontsize,
    :ytickfontsize => ytickfontsize,
    :xguidefontsize => xguidefontsize,
    :yguidefontsize => yguidefontsize,
    :legendfontsize => legendfontsize,
    :background_color_subplot => RGBA(1.0,1.0,1.0,0.9),
)

In [ ]:
println(results_0.times[end])
println(results_1.times[end])
println(results_2.times[end])
println(results_3.times[end])
println(results_4.times[end])

In [ ]:
function indice_stepvalue(x, step)
    target = 0.
    ret = Array{Int}([])
    for (i, v) in enumerate(x)
        if v >= target
            push!(ret, i)
            target += step
        end
    end
    ret
end

@recipe function f(::Type{Val{:samplemarkers}}, x, y, z; step=10)
    ind = indice_stepvalue(x, step)
    n = length(ind)
    sx, sy = x[ind], y[ind]
    # add an empty series with the correct type for legend markers
    @series begin
        seriestype := :path
        markershape --> :auto
        x := []
        y := []
    end
    # add a series for the line
    @series begin
        primary := false # no legend entry
        markershape := :none # ensure no markers
        seriestype := :path
        seriescolor := get(plotattributes, :seriescolor, :auto)
        x := x
        y := y
    end
    # return  a series for the sampled markers
    primary := false
    seriestype := :scatter
    markershape --> :auto
    x := sx
    y := sy
end

In [ ]:
# MetricLP vs Time

plot(results_2.times, results_2.metricLPs, label="PDHG",
    st=:samplemarkers, step=markerstep, shape=:auto,  # Markers
    yaxis=:log, color="black", xlim=[0, 3000], 
    xlabel="Time (seconds)", ylabel="LP Metric", title=figuretitle,
    ;plotconfig...
)
plot!(results_3.times, results_3.metricLPs, label="SPDHG (blocksize=50)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="green",
)
plot!(results_4.times, results_4.metricLPs, label="PURE-CD (blocksize=1)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="orange",
)
plot!(results_0.times, results_0.metricLPs, label="CLVR (blocksize=1)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="red",
)
plot!(results_1.times, results_1.metricLPs, label="CLVR (blocksize=50)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="blue",
)

savefig("$(savedir)/$(dataset)-MetricLP_Time-01234.png")


In [ ]:
# MetricLP vs Datapass

loggingfreq = 5
markerstep = 500

plot(collect(0:length(results_2.metricLPs)-1) * loggingfreq, results_2.metricLPs, label="PDHG",
    st=:samplemarkers, step=markerstep, shape=:auto,  # Markers
    yaxis=:log, color="black", xlim=[0, 1e4],
    xlabel="Number of data passes", ylabel="LP Metric", title=figuretitle,
    ;plotconfig...
)
plot!(collect(0:length(results_3.metricLPs)-1) * loggingfreq, results_3.metricLPs, label="SPDHG (blocksize=50)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="green",
)
plot!(collect(0:length(results_4.metricLPs)-1) * loggingfreq, results_4.metricLPs, label="PURE-CD (blocksize=1)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="orange",
)
plot!(collect(0:length(results_0.metricLPs)-1) * loggingfreq, results_0.metricLPs, label="CLVR (blocksize=1)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="red",
)
plot!(collect(0:length(results_1.metricLPs)-1) * loggingfreq, results_1.metricLPs, label="CLVR (blocksize=50)",
    st=:samplemarkers, step=markerstep, shape=:auto,
    color="blue",
)

savefig("$(savedir)/$(dataset)-MetricLP_Datapass-01234.png")


-----